<a href="https://colab.research.google.com/github/Aum-20/Aum_idenhq/blob/main/Aum_Chhajed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 19.3 MB/s eta 0:00:00


In [7]:
!playwright install chromium

164.9 MiB [] 0% 0.0s164.9 MiB [] 0% 28.5s164.9 MiB [] 0% 8.0s164.9 MiB [] 0% 6.1s164.9 MiB [] 1% 4.2s164.9 MiB [] 1% 4.1s164.9 MiB [] 2% 3.9s164.9 MiB [] 2% 3.7s164.9 MiB [] 3% 3.7s164.9 MiB [] 4% 3.5s164.9 MiB [] 4% 3.3s164.9 MiB [] 5% 3.1s164.9 MiB [] 6% 3.0s164.9 MiB [] 6% 3.3s164.9 MiB [] 6% 3.7s164.9 MiB [] 6% 3.8s164.9 MiB [] 6% 4.0s164.9 MiB [] 7% 4.1s164.9 MiB [] 7% 4.2s164.9 MiB [] 8% 4.2s164.9 MiB [] 8% 4.0s164.9 MiB [] 9% 4.0s164.9 MiB [] 9% 4.1s164.9 MiB [] 10% 4.1s164.9 MiB [] 10% 4.0s164.9 MiB [] 11% 4.0s164.9 MiB [] 11% 3.9s164.9 MiB [] 12% 3.9s164.9 MiB [] 13% 3.8s164.9 MiB [] 14% 3.8s164.9 MiB [] 14% 3.9s164.9 MiB [] 15% 3.8s164.9 MiB [] 16% 3.7s164.9 MiB [] 17% 3.6s164.9 MiB [] 18% 3.5s164.9 MiB [] 19% 3.5s164.9 MiB [] 20% 3.4s164.9 MiB [] 21% 3.3s164.9 MiB [] 22% 3.3s164.9 MiB [] 23% 3.2s164.9 MiB [] 24% 3.2s164.9 MiB [] 25% 3.2s164.9 MiB [] 25% 3.1s164.9 MiB [] 26% 3.0s164.9 MiB [] 27% 2.9s164.9 MiB [] 27% 3.0s164.9 MiB [] 28% 2.9s164.9 MiB [] 29% 2.8s164.9 MiB [] 3

In [9]:
!pip install playwright python-dotenv


In [51]:
import asyncio
import json
import os
from playwright.async_api import async_playwright, TimeoutError
import nest_asyncio

nest_asyncio.apply()

async def main():
    session_file = "session.json"
    output_file = "product_data.json"

    email = "chhajed.aum@gmail.com"
    password = "JGSMDiux"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()

        session_exists = False
        if os.path.exists(session_file):
            try:
                with open(session_file, "r") as f:
                    storage_state = json.load(f)
                    await context.set_storage_state(storage_state)
                    session_exists = True
                    print("✅ Loaded existing session.")
            except Exception as e:
                print(f"⚠ Error loading session: {e}")

        page = await context.new_page()
        print("🌐 Navigating to homepage...")
        await page.goto("https://hiring.idenhq.com")

        if not session_exists or await page.is_visible("text=Sign in"):
            print("🔐 Logging in...")
            await page.wait_for_selector("input[type='email']", timeout=120000)
            await page.fill("input[type='email']", email)
            await page.fill("input[type='password']", password)
            await page.click("button:has-text('Sign in')")
            await page.wait_for_load_state("networkidle")
            storage_state = await context.storage_state()
            with open(session_file, "w") as f:
                json.dump(storage_state, f)
            print("💾 Session saved.")

        try:
            print("🚀 Launching challenge...")
            await page.click("button:has-text('Launch Challenge')", timeout=120000)
            print("📂 Opening dashboard menu...")
            await page.click("button:has-text('Open Dashboard Menu')", timeout=120000)
            print("🛠 Clicking 'Data Tools'...")
            await page.click("text=Data Tools", timeout=120000)
            await page.click("text=Data Tools", timeout=120000)
            print("📦 Accessing Inventory Options...")
            await page.wait_for_selector("button:has-text('Inventory Options')", timeout=120000)
            await page.click("button:has-text('Inventory Options')")
            print("🛍 Opening Products Drawer...")
            await page.click("text=Open Products Drawer", timeout=120000)
            await page.wait_for_timeout(2000)
            await page.wait_for_selector("div.grid", timeout=300000)
        except TimeoutError as te:
            await page.screenshot(path="timeout_error.png")
            print(f"⛔ Timeout occurred: {te}")
            await browser.close()
            return

        async def extract_card_products():
            products = []
            product_cards = await page.query_selector_all("div.grid > div")
            print(f"🧾 Found {len(product_cards)} product cards.")

            with open("raw_grid_dump.txt", "w", encoding="utf-8") as f:
                for idx, card in enumerate(product_cards):
                    try:
                        full_text = await card.inner_text()
                        f.write(f"\n--- Card {idx+1} ---\n{full_text}\n")
                        print(f"\n🔹 Card {idx+1} Text:\n{full_text}\n")

                        lines = full_text.strip().split('\n')
                        if len(lines) >= 4:
                            product = {
                                "ID": lines[0],
                                "Dimensions": lines[1],
                                "Details": lines[2],
                                "Type": lines[3]
                            }
                            products.append(product)
                    except Exception as e:
                        print(f"⚠ Error processing card {idx+1}: {e}")
                        continue

            return products

        print("🔍 Extracting products from cards...")
        all_products = await extract_card_products()

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(all_products, f, indent=2)
        print(f"✅ Extracted {len(all_products)} products to {output_file}")

        await browser.close()

await main()


⚠ Error loading session: 'BrowserContext' object has no attribute 'set_storage_state'
🌐 Navigating to homepage...
🔐 Logging in...
💾 Session saved.
🚀 Launching challenge...
📂 Opening dashboard menu...
🛠 Clicking 'Data Tools'...
📦 Accessing Inventory Options...
🛍 Opening Products Drawer...
🔍 Extracting products from cards...
🧾 Found 2 product cards.

🔹 Card 1 Text:
Step 3 completed


🔹 Card 2 Text:
Continue to the next step.

✅ Extracted 0 products to product_data.json
